In [1]:
# move workidir to the correct folder
import os
os.chdir("../")

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import pytorch_lightning as pl

from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# Custom Libs
from dataset.dataset import Dataset
from model.models.listings_regressor import MLPRegressor, LossAccCallback

In [3]:
# Experiments Reproducibility
from pytorch_lightning import seed_everything

seed_everything(42, workers=True)

Global seed set to 42


42

In [4]:
# Load the precleaned and spearated datset
with open(f'./dataset/train_dataset.pkl', 'rb') as f:
    train_dataset = pd.read_pickle(f)

with open(f'./dataset/test_dataset.pkl', 'rb') as f:
    test_dataset = pd.read_pickle(f)

In [5]:
_, y_train = train_dataset.get()
x_not_embeddings_train = train_dataset.getListingsNotEmbeddings()
x_embeddings_train = train_dataset.getListingsEmbeddings()
print("not_embeddings_shape", x_not_embeddings_train.shape)
print("embeddings_shape", x_embeddings_train.shape)

not_embeddings_shape (5052, 146)
embeddings_shape (5052, 1536)


In [6]:
# Apply PCA to non embeddings fields + normalization
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Note that the PCA and the scaler are kept to be reused in the test
pca = PCA(n_components=60)
scaler = StandardScaler()

x_not_embeddings_train = x_not_embeddings_train.fillna(0)
x_not_embeddings_train_scaled = scaler.fit_transform(x_not_embeddings_train)
x_not_embeddings_train_60 = pca.fit_transform(x_not_embeddings_train_scaled)

In [7]:
#  define X and y for training
X_train = np.concatenate((x_not_embeddings_train_60, x_embeddings_train), axis=1)
y_train = y_train  

In [8]:
# Load and prepare Test
_, y_test = test_dataset.get()
x_not_embeddings_test = test_dataset.getListingsNotEmbeddings()
x_embeddings_test = test_dataset.getListingsEmbeddings()
print("not_embeddings_shape", x_not_embeddings_test.shape)
print("embeddings_shape", x_embeddings_test.shape)

# PCA for dimensionality reduction
pca = PCA(n_components=60)
scaler = StandardScaler()

x_not_embeddings_test = x_not_embeddings_test.fillna(0)
x_not_embeddings_test_scaled = scaler.fit_transform(x_not_embeddings_test)
x_not_embeddings_test_60 = pca.fit_transform(x_not_embeddings_test_scaled)

X_test = np.concatenate((x_not_embeddings_test_60, x_embeddings_test), axis=1)

not_embeddings_shape (1264, 146)
embeddings_shape (1264, 1536)


In [ ]:
# Bucket generations
buckets = [(0, 3), (3, 4), (4, 4.5), (4.5, 5)]
def bucketizie(x):
    for i in range(len(buckets)):
        start, end = buckets[i]
        if (i == 0): 
            if x >= start and  x<=end: return i
        else:
            if x > start and  x<=end: return i

y_train_buckets = [bucketizie(y_train[i]) for i in range(y_train.shape[0])]
y_test_buckets = [bucketizie(y_test.values[i]) for i in range(y_test.shape[0])]


In [39]:
np.unique(y_train_buckets)

array([0, 1, 2, 3])

In [72]:
# An util function to plot the result of a trained model on the test data
from sklearn import metrics
from xgboost import XGBClassifier

def evaluate_and_display(model, x, y):
    y = y.round()

    y_hat = None
    if  type(XGBClassifier()) == type(model):
        y_hat = model.predict(x).round()
    else:
        y_hat = model.transform(x).round()

    print(y)
    print(y_hat)

    ## Confusion
    cm = metrics.confusion_matrix(y, y_hat)
    disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()

    ## ROC
    fpr, tpr, thresholds = metrics.roc_curve(y, y_hat, pos_label=2)
    auc = metrics.auc(fpr, tpr)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    print("AUC:", auc)

    # Accuracy
    print("Accuracy": metrics.accuracy_score(y, y_hat))

SyntaxError: invalid syntax (2498777796.py, line 28)

In [68]:
model = XGBClassifier(n_estimators=110, nthread=-1, max_depth = 4, seed=1729)
model.fit(X_train, y_train_buckets, eval_metric="auc")

/home/mangelini/miniconda3/envs/NLP/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=110, n_jobs=None, nthread=-1, num_parallel_tree=None,
              objective='multi:softprob', ...)

In [73]:
evaluate_and_display(model, X_test, y_test_buckets)

ValueError: Classification metrics can't handle a mix of continuous and multiclass targets